PARCIAL I - Maria Fernanda Mora Varón

Nota : El factor de expancion es F_EXP

In [2]:
#Importanto las librerias necesarias para el analisi de la GEIH
import pandas as pd
import numpy as np

In [3]:
#Importar datos 
df_emicron=pd.read_csv("https://github.com/niconomist98/DataAnalyticsUQ/raw/refs/heads/main/Datos/EMICRON/emicron_identificacion.csv", encoding="latin", sep=",")

In [4]:
df_emicron.head()

,DIRECTORIO,SECUENCIA_P,SECUENCIA_ENCUESTA,COD_DEPTO,AREA,CLASE_TE,P35,P241,MES_REF,P3031,P3032_1,P3032_2,P3032_3,P3033,P3034,P3035,P3000,GRUPOS4,GRUPOS12,F_EXP
0,7627444,1,1,44,NaN,2,2,33,ENERO,2,NaN,NaN,NaN,2,240,2,2,2,3,60.050515
1,7627446,1,1,44,NaN,2,1,31,ENERO,2,NaN,NaN,NaN,2,166,2,2,1,1,86.341075
2,7627449,1,1,68,NaN,1,2,42,ENERO,2,NaN,NaN,NaN,2,60,2,2,3,5,139.884518
3,7627453,1,2,68,NaN,1,2,41,ENERO,2,NaN,NaN,NaN,2,60,2,2,4,7,168.635440
4,7627456,1,3,68,NaN,1,1,18,ENERO,2,NaN,NaN,NaN,2,9,2,2,4,6,64.659273


In [5]:
total = df_emicron["F_EXP"].sum()
print("Población total expandida:", int(total))


Población total expandida: 5297251


In [19]:
#Primero tenemos que determinar si estan las variables que necesitamos 
df_emicron.columns

Index(['DIRECTORIO', 'SECUENCIA_P', 'SECUENCIA_ENCUESTA', 'COD_DEPTO', 'AREA',
       'CLASE_TE', 'P35', 'P241', 'MES_REF', 'P3031', 'P3032_1', 'P3032_2',
       'P3032_3', 'P3033', 'P3034', 'P3035', 'P3000', 'GRUPOS4', 'GRUPOS12',
       'F_EXP'],
      dtype='object')

PREGUNTAS : 
1. ¿Cuantos micronegocios existen en el quindio cuyos propietarios son hombres del quindio son entre 18 y 30 años ?

In [6]:
# Filtrar Quindío (COD_DEPTO=63), hombres (P35=1), edad entre 18 y 30
filtro = df_emicron[
    (df_emicron["COD_DEPTO"] == 63) &
    (df_emicron["P35"] == 1) &
    (df_emicron["P241"].between(18, 30))
]

# Calcular micronegocios expandidos
total_quindio_hombres_18_30 = filtro["F_EXP"].sum()

print("Micronegocios en Quindío con propietarios hombres de 18 a 30 años:",
      round(total_quindio_hombres_18_30))


Micronegocios en Quindío con propietarios hombres de 18 a 30 años: 1929


2. ¿cuantos micronegocios de la industria manufacturera existen en el departanmento de risaralda, cuyos propietarias son mujeres entre los 20 y 30 años?


In [ ]:
# Micronegocios de industria manufacturera en Risaralda
# con propietarias mujeres entre 20 y 30 años
#Ahora agregamos la varibale de interes que es GRUPOS4 que nos permite saber de que sector es la microempresa
#GRUPO4=2 Industria manufacturera 

#Ahora agregamos esto a nuestro codigo pero cambiamos el P35 a 2 para referirnos a mujeres y el DPTO a 66 para referirnos a Risaralda 
micronegocio_manu_FR = df_emicron[(df_emicron["COD_DEPTO"] == 66) & (df_emicron["P35"] == 2) & (df_emicron["P241"].between(20, 30)) & 
                                  (df_emicron['GRUPOS4'] == 2)]


# Calcular micronegocios con F_EXP
total_microMFR = micronegocio_manu_FR["F_EXP"].sum()

print(f"Micronegocios de la industria manufacturera en Risaralda que son de mujeres entre los 20 y 30:", int(total_microMFR))

Micronegocios de la industria manufacturera en Risaralda que son de mujeres entre los 20 y 30: 471


3. ¿Entre las cuidades de manizales, pereira y armenia. Cual de las 3 tieene mas micronegocios del sector comercio, cuyas propetarias son mujeres mayores a 40 años?

In [21]:
# Filtrar micronegocios de comercio, mujeres >40, ciudades eje cafetero
filtro_comercio = df_emicron[
    (df_emicron["COD_DEPTO"].isin([17, 66, 63])) &  # Manizales, Pereira, Armenia
    (df_emicron["P35"] == 2) &                      # Mujeres
    (df_emicron["P241"] > 40) &                     # Mayores de 40 años
    (df_emicron["GRUPOS4"] == 1)                    # Comercio
]

# Agrupar por ciudad y sumar con factor de expansión
resultado = filtro_comercio.groupby("COD_DEPTO")["F_EXP"].sum().reset_index()

# Mapeo de nombres de ciudades
mapa_ciudades = {17: "Manizales (Caldas)", 66: "Pereira (Risaralda)", 63: "Armenia (Quindío)"}
resultado["Ciudad"] = resultado["COD_DEPTO"].map(mapa_ciudades)

# Ordenar de mayor a menor
resultado = resultado.sort_values("F_EXP", ascending=False)

print("Micronegocios del sector comercio con propietarias mujeres >40 años:\n")
print(resultado)


Micronegocios del sector comercio con propietarias mujeres >40 años:

   COD_DEPTO        F_EXP               Ciudad
0         17  1003.780635   Manizales (Caldas)
2         66   703.997325  Pereira (Risaralda)
1         63   111.276961    Armenia (Quindío)


4. ¿En la cuidad de armenia, cuantos micronegocios hay cuyos propetrios son hombres que no tiene  nombre comercial?

In [15]:
# Micronegocios en Armenia con propietarios hombres y sin nombre comercial
filtro_armenia = df_emicron[
    (df_emicron["COD_DEPTO"] == 63) &   # Armenia (Quindío)
    (df_emicron["P35"] == 1) &          # Hombres
    (df_emicron["P3033"] == 2)          # Sin nombre comercial
]

# Calcular población expandida
total_armenia = filtro_armenia["F_EXP"].sum()

print("Micronegocios en Armenia con propietarios hombres y sin nombre comercial:",
      round(total_armenia))


Micronegocios en Armenia con propietarios hombres y sin nombre comercial: 17566
